In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Mars, Venus
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат) 1762 дней полета
date_launch = Time("2026-02-13", scale="utc").tdb
date_flyby_Mars = Time("2028-05-02", scale="utc").tdb
date_flyby_Venus = Time("2029-07-16", scale="utc").tdb
date_arrival = Time("2034-11-16", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Mars))
mar = Ephem.from_body(Mars, time_range(date_launch, end=date_flyby_Mars))
ven = Ephem.from_body(Venus, time_range(date_launch, end=date_flyby_Venus))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_mar = Orbit.from_ephem(Sun, mar, date_flyby_Mars)
ss_ven = Orbit.from_ephem(Sun, ven, date_flyby_Venus)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_mar)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_mar, ss_ven)
ss_trans_2, ss_target_2 = ss_mar.apply_maneuver(man_lambert_2, intermediate=True)

man_lambert_3 = Maneuver.lambert(ss_ven, ss_jup)
ss_trans_3, ss_target_3 = ss_ven.apply_maneuver(man_lambert_3, intermediate=True)

In [43]:
#1
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=270 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=290 * u.deg), color="purple", label="Transfer orbit M-E"
)

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=202 * u.deg), color="black", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [7]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=270 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [8]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=290 * u.deg), color="purple", label="Transfer orbit M-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [44]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=202 * u.deg), color="black", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [10]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_mar, v_mar = Ephem.from_body(Mars, date_flyby_Mars).rv(date_flyby_Mars)
r_ven, v_ven = Ephem.from_body(Venus, date_flyby_Venus).rv(date_flyby_Venus)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [11]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

-119782285.14412466 km
79041959.80215667 km
34284867.104739465 km

[-18.02609487 -22.19132515  -9.62019085] km / s


In [12]:
# Фазовые координаты Марса

print (f"{r_mar [0]}")
print (f"{r_mar [1]}")
print (f"{r_mar [2]}"'\n')

print (v_mar.to(u.km / u.s))

191279592.59435442 km
87243151.55805336 km
34878623.54605326 km

[-9.79496601 21.55222907 10.14952239] km / s


In [13]:
# Фазовые координаты Венеры

print (f"{r_ven [0]}")
print (f"{r_ven [1]}")
print (f"{r_ven [2]}"'\n')

print (v_ven.to(u.km / u.s))

-106617330.07940297 km
-14874195.835034434 km
67680.47154989187 km

[  4.19122029 -31.72390262 -14.5406735 ] km / s


In [14]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

722333754.618259 km
153737446.87030372 km
48315888.336591184 km

[-2.99175894 12.25666453  5.32626542] km / s


In [15]:
nh_tof_1 = date_flyby_Mars - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_mar), = iod.lambert(Sun.k, r_earth, r_mar, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=809.0000000045783>

In [16]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 7.70605492 km / s>

In [17]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [-7.50076349, -1.75545257, -0.20053842] km / s>

In [18]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end = (norm(vk_mar - v_mar)).to(u.km / u.s)
V_inf_end

<Quantity 13.98377486 km / s>

In [19]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end_vec = (vk_mar - v_mar).to(u.km / u.s)
V_inf_end_vec

<Quantity [-12.14941368,  -6.01687693,  -3.42562381] km / s>

In [20]:
nh_tof_2 = date_flyby_Venus - date_flyby_Mars # время полета Марс - Венера

(v0_mar, vk_ven), = iod.lambert(Sun.k, r_mar, r_ven, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=439.999999979475>

In [21]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_0 = (norm(v0_mar - v_mar)).to(u.km / u.s)
V_inf_end_0

<Quantity 14.05641102 km / s>

In [22]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_0_vec = (v0_mar - v_mar).to(u.km / u.s)
V_inf_end_0_vec

<Quantity [13.95258974, -0.81588294,  1.4974195 ] km / s>

In [23]:
# Разница гиперболических избытков скорости подлета и отлета 

v_gip_diff_1 = V_inf_end - V_inf_end_0
v_gip_diff_1

<Quantity -0.07263616 km / s>

In [24]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end_2 = (norm(vk_ven - v_ven)).to(u.km / u.s)
V_inf_end_2

<Quantity 18.31997913 km / s>

In [25]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end_2_vec = (vk_ven - v_ven).to(u.km / u.s)
V_inf_end_2_vec

<Quantity [17.59570836,  0.96293162, -5.00853726] km / s>

In [26]:
nh_tof_3 = date_arrival - date_flyby_Venus # время полета Венера - Юпитер

(V0_ven, Vk_Jup), = iod.lambert(Sun.k, r_ven, r_jup, nh_tof_3)
nh_tof_3

<TimeDelta object: scale='tdb' format='jd' value=1948.9999999891706>

In [27]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_2_0 = (norm(V0_ven - v_ven)).to(u.km / u.s)
V_inf_end_2_0

<Quantity 18.38923535 km / s>

In [28]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_2_vec_0 = (V0_ven - v_ven).to(u.km / u.s)
V_inf_end_2_vec_0

<Quantity [ -5.60183815,  -3.00218067, -17.25602206] km / s>

In [29]:
# Разница гиперболических избытков скорости подлета и отлета 

v_gip_diff_2 = V_inf_end_2 - V_inf_end_2_0
v_gip_diff_2

<Quantity -0.06925622 km / s>

In [30]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k = (norm(Vk_Jup - v_jup)).to(u.km / u.s)
V_inf_3_k

<Quantity 11.137628 km / s>

In [31]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k_vec = (Vk_Jup - v_jup).to(u.km / u.s)
V_inf_3_k_vec

<Quantity [-6.25699164, -9.12855212, -1.25153869] km / s>

In [32]:
time_adv = nh_tof_1 + nh_tof_2 + nh_tof_3 
time_adv

<TimeDelta object: scale='tdb' format='jd' value=3197.999999973224>

In [33]:
scalar_EM = r_earth[0]*r_mar[0] + r_earth[1]*r_mar[1] +r_earth[2]*r_mar[2] 
scalar_MV = r_ven[0]*r_mar[0] + r_ven[1]*r_mar[1] +r_ven[2]*r_mar[2]
scalar_VJ = r_ven[0]*r_jup[0] + r_ven[1]*r_jup[1] +r_ven[2]*r_jup[2]

In [34]:
fi_EM = (scalar_EM/norm(r_earth)/norm(r_mar))
fi_MV = (scalar_MV/norm(r_mar)/norm(r_ven))
fi_VJ = (scalar_VJ/norm(r_ven)/norm(r_jup))

In [35]:
np.degrees(np.arccos(fi_EM))

<Quantity 118.1198525 deg>

In [36]:
np.degrees(np.arccos(fi_MV))

<Quantity 160.9820412 deg>

In [37]:
np.degrees(np.arccos(fi_VJ))

<Quantity 174.44579323 deg>

In [38]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 7.7060549 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 5.53284024 km / s>

In [39]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 11.137628

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

15.66422189447073
17.983782441612995


<Quantity -2.31956055 km / s>

In [40]:
## параметры участка Землия-Марс 

from math import*

to_rad = pi/180

ist_an_1 = 26.5032714 * to_rad 
ist_an_2 = 269.3483750 * to_rad 

exs_EM = (norm(r_mar)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(r_mar)/norm(r_earth) * cos(ist_an_2)))
p_EM = norm(r_earth)*(1+cos(ist_an_1)*exs_EM)
a_EM = p_EM/(1 - exs_EM*exs_EM)

print (f"Эксцентриситет {exs_EM}")
print (f"Фокальный параметр {p_EM}")
print (f"Большая полуось {a_EM}")

Эксцентриситет 0.48755557969057484
Фокальный параметр 211928094.908125 km
Большая полуось 278015214.87723863 km


In [41]:
## параметры участка Марс-Венера 

ist_an_1_MV = 131.0306516 * to_rad  
ist_an_2_MV = 291.4228813 * to_rad  

exs_MV = (norm(r_ven)/norm(r_mar) - 1) / (cos(ist_an_1_MV) - (norm(r_ven)/norm(r_mar) * cos(ist_an_2_MV)))
p_MV = norm(r_mar)*(1+cos(ist_an_1_MV)*exs_MV)
a_MV = p_MV/(1 - exs_MV*exs_MV)

print (f"Эксцентриситет {exs_MV}")
print (f"Фокальный параметр {p_MV}")
print (f"Большая полуось {a_MV}")

Эксцентриситет 0.5884459396927729
Фокальный параметр 130786992.01086451 km
Большая полуось 200062283.68491668 km


In [42]:
## параметры участка Венера-Юпитер

ist_an_1_VJ = 16.8983079 * to_rad   
ist_an_2_VJ = 202.4459004 * to_rad   

exs_VJ = (norm(r_jup)/norm(r_ven) - 1) / (cos(ist_an_1_VJ) - (norm(r_jup)/norm(r_ven) * cos(ist_an_2_VJ)))
p_VJ = norm(r_ven)*(1+cos(ist_an_1_VJ)*exs_VJ)
a_VJ = p_VJ/(1 - exs_VJ*exs_VJ)

print (f"Эксцентриситет {exs_VJ}")
print (f"Фокальный параметр {p_VJ}")
print (f"Большая полуось {a_VJ}")

Эксцентриситет 0.8035858960776914
Фокальный параметр 190420705.59463012 km
Большая полуось 537532428.385791 km
